In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
items = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
submission = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')
items_cat = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
train = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
shops = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
test = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')

In [ ]:
train.head()

# Merging data into one piece

In [ ]:
df_items = pd.merge(left = items,right=items_cat,on = 'item_category_id')

In [ ]:
df_shops = pd.merge(left = train,right=shops,on = 'shop_id')

In [ ]:
df = pd.merge(left = df_shops , right = df_items,on = 'item_id')

In [ ]:
df.head()

# Understanding the data

In [ ]:
df.shape

In [ ]:
df.nunique()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.describe(include = 'object')

# Data cleaning and EDA

In [ ]:
df.columns

In [ ]:
df1=df.drop(['shop_id','item_id','item_category_id'],axis=1)

In [ ]:
df1['date'] = pd.to_datetime(df1['date'],dayfirst=True ,format = '%d.%m.%Y')

In [ ]:
df1['year'] = df1['date'].dt.year
df1['month'] = df1['date'].dt.month

In [ ]:
df1.head()

In [ ]:
df1['year'].value_counts().plot(kind = 'bar')

In [ ]:
df1['month'].value_counts().plot(kind = 'bar')

In [ ]:
#As we extracted year and month we don't need  date column any more
df_new = df1.drop('date',axis=1)

In [ ]:
df_new['year'] = df_new['year'].replace({2013:1,2014:2,2015:3})

In [ ]:
df_new['year'] = df_new['year'].astype('object')

In [ ]:
df_new['month'] =  df_new['month'].astype('object')

In [ ]:
df_new.columns

In [ ]:
features = df_new[['date_block_num','item_price','item_cnt_day','year','month']]

In [ ]:
for i in features.columns:
    for j in features.columns:
        if i != j:
            sns.scatterplot(x = df_new[i],y = df_new[j])
            plt.show()

In [ ]:
df_new.isnull().sum()

In [ ]:
for i in df_new.select_dtypes(include = 'number').columns:
    sns.boxplot(df_new[i])
    plt.show()

In [ ]:
#treating outliers 
for i in df_new.select_dtypes(include = 'number').columns:
    if i != 'item_cnt_day':
        q1 = df_new[i].quantile(0.25)
        q3 = df_new[i].quantile(0.75)
        iqr = q3-q1
        ll = q1-(1.5*iqr)
        ul = q3+(1.5*iqr)
        df_new[i] = df_new[(df_new[i]>ll)&(df_new[i]<ul)][i]


In [ ]:
df_new.isnull().sum()

In [ ]:
df_new['item_price'].skew()

In [ ]:
df_new['item_price'] = df_new['item_price'].fillna(df_new['item_price'].median())

In [ ]:
df_new['year'] = df_new['year'].astype('int64')

In [ ]:
df_new['month'] =  df_new['month'].astype('int64')

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
for i in df_new.select_dtypes('object').columns:
    df_new[i] = le.fit_transform(df_new[i])

In [ ]:
df_new.head()

# Converting the data into dependent and independent features

In [ ]:
df_new.head()

In [ ]:
df_new.info()

In [ ]:
x = df_new.drop('item_cnt_day',axis=1)
y = np.log(df_new['item_cnt_day'])
y = y.fillna(y.median())

# Model Building

In [ ]:
import statsmodels.api as sm
import sklearn.metrics
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.feature_selection import RFE

In [ ]:
xc = sm.add_constant(x)
model = sm.OLS(y,xc).fit()
print(model.summary())

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,train_size=0.7,random_state=7)

In [ ]:
lr = LinearRegression()
model_v1 = lr.fit(xtrain,ytrain)
ypred = model_v1.predict(xtest)

In [ ]:
lr_scores = cross_val_score(estimator = lr,X=x,y=y,scoring = 'neg_root_mean_squared_error',cv = 5)
lr_rmse = np.mean(lr_scores)
print('RMSE = ',lr_rmse)
print(np.std(lr_scores))

In [ ]:
rf = RFE(estimator = lr,n_features_to_select=5)
rf.fit(x,y)

In [ ]:
rfe_df = pd.DataFrame({'features':x.columns,'ranking':rf.ranking_})

In [ ]:
rfe_df

In [ ]:
# as item_price and item_name is not important we are going to drop it
x = x.drop(['item_price','item_name'],axis=1)


In [ ]:
# We can do OLS again
xc = sm.add_constant(x)
model = sm.OLS(y,xc).fit()
model.summary()

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,train_size=0.7,random_state=7)

In [ ]:
model_v2 = lr.fit(xtrain,ytrain)
ypred_lr = model_v2.predict(xtest)

In [ ]:
lr_scores = cross_val_score(estimator = lr,X=x,y=y,scoring = 'neg_root_mean_squared_error',cv = 5)
lr_rmse = np.mean(lr_scores)
print('RMSE = ',lr_rmse)
print(np.std(lr_scores))

In [ ]:
#RMSE and stability improved a little bit

In [ ]:
resid = pd.DataFrame({'Predicted' : ypred , 'Actuals':ytest,'Residual':ypred-ytest})

In [ ]:
plt.scatter(resid['Predicted'],resid['Residual'])
plt.grid()

In [ ]:
# most of the predictions have an error rate between 0 to -6

In [ ]:
rid = Ridge()
model_v3 = rid.fit(xtrain,ytrain)
ypred_rid = model_v3.predict(xtest)

In [ ]:
rid_scores = cross_val_score(estimator = rid,X=x,y=y,scoring = 'neg_root_mean_squared_error',cv = 5)
rid_rmse = np.mean(rid_scores)
print('RMSE = ',rid_rmse)
print(np.std(rid_scores))

In [ ]:
# RMSE is not much different compared to Linear Regression

In [ ]:
resid = pd.DataFrame({'Predicted' : ypred_rid , 'Actuals':ytest,'Residual':ypred_rid-ytest})
plt.scatter(resid['Predicted'],resid['Residual'])
plt.grid()

In [ ]:
# even with feature selection there is not much of an improvement

In [ ]:
# Let's move to non-linear models
x = df_new.drop('item_cnt_day',axis=1)
y = np.log(df_new['item_cnt_day'])
y = y.fillna(y.median())


In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,train_size=0.7,random_state=7)

In [ ]:
knn = KNeighborsRegressor()
model_v4 = knn.fit(xtrain,ytrain)
ypred_knn = model_v4.predict(xtest)

In [ ]:
knn_scores = cross_val_score(estimator = knn,X=x,y=y,scoring = 'neg_root_mean_squared_error',cv = 5)
knn_rmse = np.mean(knn_scores)
print('RMSE = ',knn_rmse)
print(np.std(knn_scores))

In [ ]:
# error is increased and stability stability is reduced

In [ ]:
resid = pd.DataFrame({'Predicted' : ypred_knn , 'Actuals':ytest,'Residual':ypred_knn-ytest})
plt.scatter(resid['Predicted'],resid['Residual'])
plt.grid()

In [ ]:
dt = DecisionTreeRegressor()
model_v5 = dt.fit(xtrain,ytrain)
ypred_dt = model_v5.predict(xtest)

In [ ]:
dt_scores = cross_val_score(estimator = dt,X=x,y=y,scoring = 'neg_root_mean_squared_error',cv = 5)
dt_rmse = np.mean(dt_scores)
print('RMSE = ',dt_rmse)
print(np.std(dt_scores))

In [ ]:
# here both error rate and stability is depleted

In [ ]:
resid = pd.DataFrame({'Predicted' : ypred_dt , 'Actuals':ytest,'Residual':ypred_dt-ytest})
plt.scatter(resid['Predicted'],resid['Residual'])
plt.grid()

In [ ]:
gb = GradientBoostingRegressor()
model_v6 = gb.fit(xtrain,ytrain)
ypred_gb = model_v6.predict(xtest)

In [ ]:
gb_scores = cross_val_score(estimator = gb,X=x,y=y,scoring = 'neg_root_mean_squared_error',cv = 5)
gb_rmse = np.mean(gb_scores)
print('RMSE = ',gb_rmse)
print(np.std(gb_scores))

In [ ]:
resid = pd.DataFrame({'Predicted' : ypred_gb , 'Actuals':ytest,'Residual':ypred_gb-ytest})
plt.scatter(resid['Predicted'],resid['Residual'])
plt.grid()

In [ ]:
xgb = XGBRegressor()
model_v7 = xgb.fit(xtrain,ytrain)
ypred_xgb = model_v7.predict(xtest)

In [ ]:
xgb_scores = cross_val_score(estimator = xgb,X=x,y=y,scoring = 'neg_root_mean_squared_error',cv = 5)
xgb_rmse = np.mean(xgb_scores)
print('RMSE = ',xgb_rmse)
print(np.std(xgb_scores))

In [ ]:
resid = pd.DataFrame({'Predicted' : ypred_xgb , 'Actuals':ytest,'Residual':ypred_xgb-ytest})
plt.scatter(resid['Predicted'],resid['Residual'])
plt.grid()

In [ ]:
scores_df = pd.DataFrame({'Models':['Linear Regression','Ridge','KNN','Decision Tree','Gradient Boosting','XG Boosting'],
                         'Scores_RMSE' : [lr_rmse,rid_rmse,knn_rmse,dt_rmse,gb_rmse,xgb_rmse]})

In [ ]:
scores_df.sort_values('Scores_RMSE',ascending=False)

In [ ]:
# Xg Boost is the best predictor out of all